In [49]:
#Import necessary libraries

import requests
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [50]:
html_doc=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(html_doc, 'html.parser')
tab_rows=soup.table.find_all('tr')


output_rows=[]
output_rows.append(['PostCode','Borough','Neighbourhood'])

for tr in tab_rows:
    td=tr.find_all('td')
    output_row=[]
    for i in td:
        output_row.append(i.text)
    output_rows.append(output_row)
    
with open('output.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_rows)

In [116]:
# Reading into a dataframe
df=pd.read_csv('output.csv')
df.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [124]:
# Remove the /n
df['Neighbourhood']=df['Neighbourhood'].str.strip()
df.head()

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Not assigned


In [125]:
# Remove entries with no values for Borough
df=df[df.Borough != 'Not assigned']
df.reset_index(drop=True,inplace=True)
df.head()

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [126]:
# Group with same PostCode

ind_drp=[]

for i in range(0,len(df['PostCode'])):
    for j in range(i+1,len(df['PostCode'])):
        if df['PostCode'][i]==df['PostCode'][j]:
            df['Neighbourhood'][i]=df['Neighbourhood'][i]+', '+df['Neighbourhood'][j]
            ind_drp.append(j)
            
df=df.drop(df.index[ind_drp])
df.reset_index(drop=True,inplace=True)

In [127]:
# Assign same values for missing Neighbourhood
for i in range(0,len(df['PostCode'])):
    if df['Neighbourhood'][i]=='Not assigned' :
            df['Neighbourhood'][i]=df['Borough'][i]

In [132]:
df.head()

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [133]:
df.shape

(103, 3)